In [1]:
import os

In [ ]:
os.chdir("../")

UsageError: Line magic function `%pwdy` not found.


In [ ]:
# GOING BACK TO RIGHT DIRECTORY
%pwd

'e:\\Projects\\E2E Emotion Detection from text\\Emotion-Detection-using-ML'

In [ ]:
# goes to src/entity/config_entity.py
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from src.ML_emotion_detection.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.ML_emotion_detection.utils.common import read_yaml, create_directories  
from src.ML_emotion_detection import logger

In [ ]:
# Goes to src/configuration.py
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        logger.info(f'Reading CONFIG YAML FROM {config_filepath}')
        self.config = read_yaml(config_filepath)
        
        logger.info(f'Reading PARAMS YAML FROM {params_filepath}')
        self.params = read_yaml(params_filepath)
        
        logger.info(f'Reading SCHEMA YAML FROM {schema_filepath}')
        self.schema = read_yaml(schema_filepath)

        logger.info(f'Creating directory stored at artifacts_root of CONFIG YAML')
        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        logger.info(f'Creating directory stored at root_dir of CONFIG YAML')
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [ ]:
import os
import urllib.request as request
import zipfile
from src.ML_emotion_detection.utils.common import get_size
from src.ML_emotion_detection import logger

In [ ]:
# Goes to src/components/data_ingestion.py
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-17 12:55:15,705: INFO: 1800916153: Reading CONFIG YAML FROM config\config.yaml]
[2024-11-17 12:55:15,742: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-17 12:55:15,745: INFO: 1800916153: Reading PARAMS YAML FROM params.yaml]
[2024-11-17 12:55:15,802: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-17 12:55:15,803: INFO: 1800916153: Reading SCHEMA YAML FROM schema.yaml]
[2024-11-17 12:55:15,808: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-17 12:55:15,809: INFO: 1800916153: Creating directory stored at artifacts_root of CONFIG YAML]
[2024-11-17 12:55:15,844: INFO: common: created directory at: artifacts]
[2024-11-17 12:55:15,844: INFO: 1800916153: Creating directory stored at root_dir of CONFIG YAML]
[2024-11-17 12:55:15,856: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-17 12:55:21,077: INFO: 2280763589: artifacts/data_ingestion/data.zip download! with following info: 
Connection: clo